# Mangling Log Files with Regexp

In [4]:
!cat DataSets/LogDB.out | head

192.168.13.72 - - [08/May/2015:12:00:00 -0000] "POST /doc/0 HTTP/1.1" 200 138 0.155
192.168.13.85 - - [08/May/2015:12:00:00 -0000] "PUT /doc/1 HTTP/1.1" 200 265 11.933
192.168.13.75 - - [08/May/2015:12:00:00 -0000] "POST /doc/2 HTTP/1.1" 200 138 1.190
192.168.13.85 - - [08/May/2015:12:00:01 -0000] "PUT /doc/1 HTTP/1.1" 200 265 34.552
192.168.13.13 - - [08/May/2015:12:00:01 -0000] "GET /doc/3 HTTP/1.1" 200 4047 0.394
192.168.13.13 - - [08/May/2015:12:00:01 -0000] "GET /doc/4 HTTP/1.1" 200 795 0.080
192.168.13.85 - - [08/May/2015:12:00:01 -0000] "PUT /doc/1 HTTP/1.1" 200 265 11.649
192.168.13.66 - - [08/May/2015:12:00:01 -0000] "GET /doc/3 HTTP/1.1" 200 4047 0.351
192.168.13.66 - - [08/May/2015:12:00:01 -0000] "GET /doc/4 HTTP/1.1" 200 795 0.068
192.168.13.85 - - [08/May/2015:12:00:01 -0000] "PUT /doc/1 HTTP/1.1" 200 265 6.433
cat: write error: Broken pipe


Convert to csv file with fields
* method
* url
* response
* latency

In [18]:
# Capture "-"
!cat DataSets/LogDB.out | perl -pe 's/^.*"(.*)" (.*)$/\1,\2/' | head

POST /doc/0 HTTP/1.1,200 138 0.155
PUT /doc/1 HTTP/1.1,200 265 11.933
POST /doc/2 HTTP/1.1,200 138 1.190
PUT /doc/1 HTTP/1.1,200 265 34.552
GET /doc/3 HTTP/1.1,200 4047 0.394
GET /doc/4 HTTP/1.1,200 795 0.080
PUT /doc/1 HTTP/1.1,200 265 11.649
GET /doc/3 HTTP/1.1,200 4047 0.351
GET /doc/4 HTTP/1.1,200 795 0.068
PUT /doc/1 HTTP/1.1,200 265 6.433
-p destination: Broken pipe
cat: write error: Broken pipe


In [19]:
# Split "\1" into three groups separated by space
!cat DataSets/LogDB.out | perl -pe 's/^.*"(.*?) (.*)? (.*)?"(.*)$/\1,\2,\4/' | head

POST,/doc/0, 200 138 0.155
PUT,/doc/1, 200 265 11.933
POST,/doc/2, 200 138 1.190
PUT,/doc/1, 200 265 34.552
GET,/doc/3, 200 4047 0.394
GET,/doc/4, 200 795 0.080
PUT,/doc/1, 200 265 11.649
GET,/doc/3, 200 4047 0.351
GET,/doc/4, 200 795 0.068
PUT,/doc/1, 200 265 6.433
-p destination: Broken pipe
cat: write error: Broken pipe


In [28]:
# Split \4 into number groups separated by space
!cat DataSets/LogDB.out | perl -pe 's/^.*"(.*?) (.*)? (.*)?" (\d+) (\d+) (\d+.\d+)$/\1,\2,\4,\5,\6/' | head

POST,/doc/0,200,138,0.155
PUT,/doc/1,200,265,11.933
POST,/doc/2,200,138,1.190
PUT,/doc/1,200,265,34.552
GET,/doc/3,200,4047,0.394
GET,/doc/4,200,795,0.080
PUT,/doc/1,200,265,11.649
GET,/doc/3,200,4047,0.351
GET,/doc/4,200,795,0.068
PUT,/doc/1,200,265,6.433
-p destination: Broken pipe
cat: write error: Broken pipe


In [29]:
!cat DataSets/LogDB.out | perl -pe 's/^.*"(.*?) (.*)? (.*)?" (\d+) (\d+) (\d+.\d+)$/\1,"\2","\4",\5,\6/'\
> DataSets/LogDB.csv

In [33]:
!echo "method,path,code,size,latency" > DataSets/LogDB_head.csv

In [52]:
%%bash

# Add headder
cat <(echo "method,path,code,size,latency") DataSets/LogDB.csv > DataSets/LogDB_full.csv

# View Statistics
cat DataSets/LogDB_full.csv  | csvstat

  1. method
	<type 'unicode'>
	Nulls: False
	Values: PUT, POST, GET
  2. path
	<type 'unicode'>
	Nulls: False
	Unique values: 2171
	5 most frequent values:
		/doc/1:	2440
		/doc/0:	761
		/doc/18:	758
		/doc/3:	500
		/doc/4:	454
	Max length: 9
  3. code
	<type 'int'>
	Nulls: False
	Values: 200, 500, 404, 413
  4. size
	<type 'int'>
	Nulls: False
	Min: 59
	Max: 598859
	Sum: 21067128
	Mean: 2420.39613971
	Median: 265.0
	Standard Deviation: 9173.22067508
	Unique values: 966
	5 most frequent values:
		265:	2414
		138:	1265
		795:	454
		267:	270
		166:	268
  5. latency
	<type 'float'>
	Nulls: False
	Min: 0.048
	Max: 204250.748
	Sum: 26845591.645
	Mean: 3084.28212833
	Median: 7.8515
	Standard Deviation: 13668.5665645
	Unique values: 6667
	5 most frequent values:
		0.067:	35
		0.071:	28
		0.077:	25
		0.072:	23
		0.069:	22

Row count: 8704


In [59]:
# plot histogram

!csvcut -c "latency" DataSets/LogDB_full.csv | feedgnuplot --line --histogram 0 --with boxes

^C

